In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import chromedriver_binary
import requests
import pandas as pd

#ヘッドレス設定
from selenium.webdriver.chrome.options import Options
op = Options()
op.add_argument("--disable-gpu");
op.add_argument("--disable-extensions");
op.add_argument("--proxy-server='direct://'");
op.add_argument("--proxy-bypass-list=*");
op.add_argument("--start-maximized");
op.add_argument("--headless");

browser = webdriver.Chrome(options=op)

country_list = [['日本','japan'],['中国','china']]

name_data = []
code_data = []
price_data = []
data = []
    
for country in country_list:
    url = f'https://www.onlineidd.com/products/en/fln/{country[1]}/'
    browser.get(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    product_lists = soup.find_all('table', attrs={'class': 'product_list'})
    for product_list in product_lists:
        item_descriptions = product_list.find_all('td', attrs={'class': 'item_description'})

        for item_description in item_descriptions:
            item_name = item_description.find('a')
            name_data.append(item_name.text)

        codes = product_list.find_all('span', attrs={'class': 'silver'})
        for code in codes:
            tmp = code.text.replace('\r\n\t\tIntercat code: ', '')
            code_data.append(tmp)

        item_tds = product_list.find_all('td', attrs={'class': 'item'})
        for item_td in item_tds:
            price = item_td.find('b')
            price = price.text.replace('\r\n\t\t\tFLN ','')
            if price != 'Description:':
                price_data.append(price)

    for i in range(len(code_data)):
        sub = []
        sub.append(code_data[i])
        sub.append(name_data[i])
        sub.append(price_data[i]) 
        sub.append(country[0])
        data.append(sub)

df = pd.DataFrame(data, columns=['ID','商品名','価格','国'])
df
df.to_csv('商品集計.csv', index=False)

browser.quit()